<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

This notebook uses the [Wine Reviews](https://www.kaggle.com/zynicide/wine-reviews) dataset. Execute the following commands to download the data into the correct directory.
~~~~
cd ../Data
kaggle datasets download -d zynicide/wine-reviews
~~~~

# Import Pakcages

In [4]:
import pandas as pd
import pandas_profiling as pp

# Import Data

In [5]:
reviews = pd.read_csv( '../Data/wine-reviews/winemag-data-130k-v2.csv', index_col=0)

In [6]:
reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [7]:
pp.ProfileReport( reviews )

/Users/josh/opt/anaconda3/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,13
Number of observations,129971
Total Missing (%),12.1%
Total size in memory,13.9 MiB
Average record size in memory,112.0 B
Numeric,2
Categorical,11
Boolean,0
Date,0
Text (Unique),0
Rejected,0


# Experiment with Grouping

In [29]:
reviews.groupby( ['country', 'province'] ).description.agg( [len] ).sort_values( by='len').reset_index()

,country,province,len
0,Greece,Muscat of Kefallonian,1
1,Greece,Sterea Ellada,1
2,Greece,Thraki,1
3,South Africa,Paardeberg,1
4,Brazil,Serra do Sudeste,1
...,...,...,...
420,US,Oregon,5373
421,Italy,Tuscany,5897
422,France,Bordeaux,5941
423,US,Washington,8639


What is the best wine I can buy for a given amount of money? Create a `Series` whose index is wine prices and whose values is the maximum number of points a wine costing that much was given in a review. Sort the values by price, ascending (so that `4.0` dollars is at the top and `3300.0` dollars is at the bottom). 

In [31]:
# index is wine prices
# values is the max points for indexed price
# sort values by price ascending
best_rating_per_price = reviews.groupby( 'price').points.max()
best_rating_per_price

price
4.0       86
5.0       87
6.0       88
7.0       91
8.0       91
          ..
1900.0    98
2000.0    97
2013.0    91
2500.0    96
3300.0    88
Name: points, Length: 390, dtype: int64

What are the minimum and maximum prices for each `variety` of wine? Create a `DataFrame` whose index is the `variety` category from the dataset and whose values are the `min` and `max` values thereof. 

In [35]:
price_extremes = reviews.groupby( 'variety' ).price.agg( [min, max] )
price_extremes

,min,max
variety,,
Abouriou,15.0,75.0
Agiorgitiko,10.0,66.0
Aglianico,6.0,180.0
Aidani,27.0,27.0
Airen,8.0,10.0
...,...,...
Zinfandel,5.0,100.0
Zlahtina,13.0,16.0
Zweigelt,9.0,70.0


What are the most expensive wine varieties? Create a variable `sorted_varieties` containing a copy of the dataframe from the previous question where varieties are sorted in descending order based on minimum price, then on maximum price (to break ties).

In [38]:
sorted_varieties = price_extremes.sort_values( ['min', 'max'], ascending=False )
sorted_varieties

,min,max
variety,,
Ramisco,495.0,495.0
Terrantez,236.0,236.0
Francisa,160.0,160.0
Rosenmuskateller,150.0,150.0
Tinta Negra Mole,112.0,112.0
...,...,...
Roscetto,NaN,NaN
Sauvignon Blanc-Sauvignon Gris,NaN,NaN
Tempranillo-Malbec,NaN,NaN


Create a `Series` whose index is reviewers and whose values is the average review score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns. 

In [40]:
reviewer_mean_ratings = reviews.groupby( 'taster_name' ).points.mean()
reviewer_mean_ratings

taster_name
Alexander Peartree    85.855422
Anna Lee C. Iijima    88.415629
Anne Krebiehl MW      90.562551
Carrie Dykes          86.395683
Christina Pickard     87.833333
Fiona Adams           86.888889
Jeff Jenssen          88.319756
Jim Gordon            88.626287
Joe Czerwinski        88.536235
Kerin O’Keefe         88.867947
Lauren Buzzeo         87.739510
Matt Kettmann         90.008686
Michael Schachner     86.907493
Mike DeSimone         89.101167
Paul Gregutt          89.082564
Roger Voss            88.708003
Sean P. Sullivan      88.755739
Susan Kostrzewa       86.609217
Virginie Boone        89.213379
Name: points, dtype: float64

What combination of countries and varieties are most common? Create a `Series` whose index is a `MultiIndex`of `{country, variety}` pairs. For example, a pinot noir produced in the US should map to `{"US", "Pinot Noir"}`. Sort the values in the `Series` in descending order based on wine count.

In [55]:
country_variety_counts = reviews.groupby( ['country', 'variety'] ).size().sort_values( ascending=False )
country_variety_counts

country    variety                 
US         Pinot Noir                  9885
           Cabernet Sauvignon          7315
           Chardonnay                  6801
France     Bordeaux-style Red Blend    4725
Italy      Red Blend                   3624
                                       ... 
Uruguay    Tempranillo-Tannat             1
Italy      Pignolo                        1
           Muscat                         1
           Moscato di Noto                1
Argentina  Barbera                        1
Length: 1612, dtype: int64

In [80]:
reviews_per_region = reviews.region_1.fillna( 'unknown' ).value_counts()
reviews_per_region

unknown                         21247
Napa Valley                      4480
Columbia Valley (WA)             4124
Russian River Valley             3091
California                       2629
                                ...  
McLaren Vale-Langhorne Creek        1
Saint-Chinian-Roquebrun             1
Côtes de Saint-Mont                 1
Coteaux du Lyonnais                 1
Offida Rosso                        1
Name: region_1, Length: 1230, dtype: int64